In [21]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
from scipy import signal
from tensorflow.python.ops import nn_ops, array_ops

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

#Wraper podstawiajacy 
def py_func(func, inp, Tout, stateful=True, name=None):
    
    # Nowa nazwa, żeby móc modyfikować nasz gradient
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))
    tf.RegisterGradient(rnd_name)(_BinGrad)
    
    #Podstawienie gradientu
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)

#Wrapper na wrappera
def custom_conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID', name='MySquare'):
    return py_func(conv, [x,W], [tf.float32])[0]
    
#Skopiowany gradient z conv2d
def _BinGrad(op, grad):
    return [nn_ops.conv2d_backprop_input(
    array_ops.shape(op.inputs[0]), op.inputs[1], grad,(1,1,1,1),
    'VALID', False, 'NHWC'),
      nn_ops.conv2d_backprop_filter(op.inputs[0],
                                    array_ops.shape(op.inputs[1]), grad,
                                    (1,1,1,1),
                                    'VALID',
                                    False,
                                    'NHWC')]

#Nasza operacja
def conv(x, W, strides=[1, 1, 1, 1], padding='VALID'):
    x_height = x.shape[1]
    x_width = x.shape[2]
    W_height = W.shape[0]
    W_width = W.shape[1]
    filters = W.shape[3]
    inp = x[0,0:x_height,0:x_width,0]
    fil = np.rot90(W[0:W_height,0:W_width,0,0], 2)
    result = signal.convolve2d(inp, fil, 'valid')
    return result.reshape((1,2,2,1))
    

x = tf.placeholder(tf.float32, [None, 9])
x_reshaped = tf.reshape(x, [-1,3,3,1])
w = np.zeros([2,2,1,1])
w[0:2,0:2,0,0] = np.vstack(([0.1,0.2],[1,0.3]))
W_conv1 = tf.Variable(tf.cast(w,dtype=tf.float32))

sess = tf.Session()
sess.as_default()
sess.run(tf.global_variables_initializer())

numbers = np.reshape(np.array([2,1,1,1,1,1,1,2,1], dtype=np.float32),(1,9))
result = custom_conv2d(x_reshaped, W_conv1)
result_n = conv2d(x_reshaped, W_conv1)
result_conv = result.eval(session=sess, feed_dict={x: numbers})
result_norm = result_n.eval(session=sess, feed_dict={x: numbers})
print(result_conv)
print('\n')
print(result_norm)



[[[[ 1.70000005]
   [ 1.60000002]]

  [[ 1.9000001 ]
   [ 2.5999999 ]]]]


[[[[ 1.69999993]
   [ 1.5999999 ]]

  [[ 1.9000001 ]
   [ 2.5999999 ]]]]


In [20]:
var_grad = tf.gradients(result, W_conv1)[0]
var_grad_n = tf.gradients(result_n, W_conv1)[0]
gradients = var_grad.eval(session=sess, feed_dict={x: np.reshape(numbers,(1,9))})
gradients_n = var_grad_n.eval(session=sess, feed_dict={x: np.reshape(numbers,(1,9))})
print(gradients)
print('\n')
print(gradients_n)

[[[[ 5.]]

  [[ 4.]]]


 [[[ 5.]]

  [[ 5.]]]]


[[[[ 5.]]

  [[ 4.]]]


 [[[ 5.]]

  [[ 5.]]]]
